<a href="https://colab.research.google.com/github/ProfAI/tutorials/blob/master/nn_with_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Una Rete Neurale da Zero

In [0]:
import numpy as np

#### Metriche

## Accuracy

$$ \frac{1}{N}\sum_{i=0}^N(\hat{y}-y)^2 $$

In [0]:
def accuracy(y, y_pred):
  return np.sum(y==y_pred)/len(y)

### Log Loss
$$ -\frac{1}{N}\sum_{i=0}^N(y\cdot log(a) + (1-y)\cdot(1-a) $$

In [0]:
def log_loss(y_true, y_proba):
  return -np.sum(y_true*np.log(y_proba)+(1-y_true)*np.log(1-y_proba))/len(y_true)

### Predizione

$$ z_1 = W_1x+b_1 $$
<br/>
$$ a_1 = \text{relu}(z_1) $$
<br/>
$$ z_2 = W_2a_1+b_2 $$
<br/>
$$ a_2 = \text{sigmoid}(z_2) $$
<br/>
$$ \hat{y} = \begin{Bmatrix} 
1  \ \ \ se \ \  a_1\geq 0.5
\\ 
0  \ \ \ se \ \  a_1<0.5
\end{Bmatrix}
$$

$$ \text{relu}(z) = \begin{Bmatrix} 
0  \ \ \ se \ \  z < 0
\\ 
z  \ \ \ se \ \ z \geq 0
\end{Bmatrix}
$$
<br>
$$ \text{sigmoid(z)} = \frac{1}{1+e^{-z}} $$

In [0]:
def relu(Z):
  return np.maximum(Z, 0)

def sigmoid(Z):
  return 1/(1+np.power(np.e,-Z))

In [0]:
Z1=0
A1=0 
A2=0

def forward_propagation(X, W1, b1, W2, b2):
  
  global A1
  global A2
  global Z1
  
  Z1 = np.dot(X,W1)+b1
  
  A1 = relu(Z1)
  Z2 = np.dot(A1,W2)+b2
  A2 = sigmoid(Z2)
  
  return A2.ravel()


def predict(X, W1, b1, W2, b2):
  proba = forward_propagation(X, W1, b1, W2, b2)
  y = proba.copy()
  y[y>=0.5]=1
  y[y<0.5]=0
  return y

### Addestramento

In [0]:
def init_weights(input_size, hidden_units):
  
  W1 = np.random.rand(input_size, hidden_units)
  b1 = np.zeros(hidden_units)
  W2 = np.random.rand(hidden_units,1)
  b2 = np.zeros(1)

  return W1, b1, W2, b2

$$
\text{relu_derivative}(z) =  \begin{Bmatrix} 
0  \ \ \ se \ \  z \leq 0
\\ 
1  \ \ \ se \ \ z > 0
\end{Bmatrix}
$$

In [0]:
def relu_derivative(Z):
  Z[Z<=0] = 0
  Z[Z>0] = 1
  return Z

In [0]:
def fit(X, hidden, epochs=100, lr=0.01):
  
  W1, b1, W2, b2 = init_weights(X.shape[1], hidden)
   
  for _ in range(epochs):
    Y = predict(X, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = back_propagation(X, W2, A1, A2, Y)
    W1+=lr*dW1
    b1+=lr*db1
    W2+=lr*dW2
    b2+=lr*db2
    
  return W1, b1, W2, b2
    
  
def back_propagation(X, W2, A1, A2, Y):
  
  m = A1.shape[1]
  dZ2 = A2.reshape(-1,1)-Y.reshape(-1,1)
  dW2 = np.dot(A1.T, dZ2)/m
  db2 = np.sum(dZ2, axis=0)
  
  m = X.shape[1]
  dZ1 = np.dot(dZ2.reshape(-1,1), W2.T)*relu_derivative(Z1)
  dW1 = np.dot(X.T, dZ1)/m
  db1 = np.sum(dZ2, axis=0)
  
  return dW1, db1, dW2, db2

## Testing

In [0]:
from sklearn.datasets import load_breast_cancer

In [0]:
dataset = load_breast_cancer()

X = dataset.data
y = dataset.target

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

### Scikit Learn

In [23]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=(100, ))
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

In [27]:
from sklearn.metrics import accuracy_score, log_loss

print("Accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Log Loss: %.4f" % log_loss(y_test, y_proba))

Accuracy: 0.92
Log Loss: 0.2503


### La nostra Rete Neurale

In [0]:
W1, b1, W2, b2 = fit(X_train, 100, epochs=1)

In [37]:
y_pred = predict(X_test, W1, b1, W2, b2)
accuracy(y_test, y_pred)

0.631578947368421

## La Rete Neurale in una Classe

In [0]:
class NeuralNetwork:
  
  
  def __init__(self, hidden_layer_size=100):
    
    self.hidden_layer_size=hidden_layer_size
    
    
  def _init_weights(self, input_size, hidden_size):
    
    self._W1 = np.random.rand(input_size, hidden_size)
    self._b1 = np.zeros(hidden_size)
    self._W2 = np.random.rand(hidden_size,1)
    self._b2 = np.zeros(1)

    
  def _accuracy(self, y, y_pred):
               
    return np.sum(y==y_pred)/len(y)
    
               
  def _forward_propagation(self, X):
               
    Z1 = np.dot(X,self._W1)+self._b1

    A1 = relu(Z1)
    Z2 = np.dot(A1,self._W1.T)+self._b2
    A2 = sigmoid(Z2)

    self._forward_cache = (Z1, A1, Z2, A2)

    return A2.ravel()


  def predict(self, X):

      proba =self._forward_propagation(X)

      y = proba
      y[y>=0.5]=1
      y[y<0.5]=0

      return y


  def predict_proba(self, X):         
      return _forward_propagation(X)
                            
      
  def _back_propagation(self, X, y):
  
    Z1, A1, Z2, A2 = self._forward_cache
    
               
    m = A1.shape[1]
    dZ2 = A2.reshape(-1,1)-y.reshape(-1,1)
    dW2 = np.dot(A1.T, dZ2)/m
    db2 = np.sum(dZ2, axis=0)

    m = X.shape[1]
    dZ1 = np.dot(dZ2.reshape(-1,1), W2.T)*relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)/m
    db1 = np.sum(dZ2, axis=0)

    return dW1, db1, dW2, db2
           
               
  def fit(self, X, y, epochs=100, lr=0.01):
     
    self._init_weights(X.shape[1], self.hidden_layer_size)
      
    for _ in range(epochs):
      Y = self._forward_propagation(X)
      dW1, db1, dW2, db2 = self._back_propagation(X, y)
      self._W1+=lr*dW1
      self._b1+=lr*db1
      self._W2+=lr*dW2
      self._b2+=lr*db2
               

  def evaluate(X, y):
    y_pred = self.predict(X)
    return self.accuracy(y, y_pred)

In [56]:
model = NeuralNetwork()
model.fit(X_train, y_train)
model.evaluate(X_test, y_test)

ValueError: ignored